# <center> Python Client Example
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" /> 
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2021

# Web Clients
While often we think of a web client to be a browser, we can also use programs or libraries to access web based resources. 

Python has the requests library for these transactions.

https://requests.readthedocs.io/en/master/

Run this notebook after the Tornado server notebook is running.

In [1]:
import requests
import base64
import json

In [2]:
url = "http://localhost:8080"
r = requests.get(url)
print(r)

<Response [200]>


What are the status codes?

https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

200 is good.

In [3]:
#Responses have status codes
r.status_code

200

In [4]:
# Text base response
r.text

'{"message": "The current time is 1632450248.170048"}'

In [5]:
# Bytes as the response. Use this to read binary (like photos)
r.content 

b'{"message": "The current time is 1632450248.170048"}'

In [6]:
# Access lower level bytes from the socket
r = requests.get(url, stream=True)
r.raw

In [7]:
r.raw.read()

b'{"message": "The current time is 1632450248.2390492"}'

In [8]:
# Headers are often not displayed to the client, but are available
# requests makes it a dictionary. 
# This is the header response from the server
r.headers

{'Server': 'TornadoServer/6.0.3', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Fri, 24 Sep 2021 02:24:08 GMT', 'Etag': '"ce288ec10effe4ab1ec3f70df02b1c23579b3a6a"', 'Content-Length': '53'}

In [9]:
r.encoding

'UTF-8'

In [10]:
r.url

'http://localhost:8080/'

In [11]:
#This is actualy json, so we can load it directly into a dictionary
r = requests.get(url)
print(r.status_code)
r.json()

200


{'message': 'The current time is 1632450248.3392925'}

In [12]:
plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."
plain_text

'I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.'

In [13]:
#Get the key (Don't actually do this)
r = requests.get(url+"/encrypted/")
print(r.status_code)
print(r.text)
pub_key_text = r.json()['pub']
print(pub_key_text)

200
{"pub": "-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEAVDIB4f2hbUmXtDnew9hPEsaY6iOvK0B7R+PCKTSR1Qs=\n-----END PUBLIC KEY-----\n"}
-----BEGIN PUBLIC KEY-----
MCowBQYDK2VuAyEAVDIB4f2hbUmXtDnew9hPEsaY6iOvK0B7R+PCKTSR1Qs=
-----END PUBLIC KEY-----



In [14]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives import serialization
#Client needs to generate a key pair
private_key_for_client= X25519PrivateKey.generate()
private_key_for_client

In [15]:
client_public_pem = private_key_for_client.public_key().public_bytes(
       encoding=serialization.Encoding.PEM,
       format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print(client_public_pem)

b'-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEA1QOfOT1wC4o3zabkRMDczxSBDyHOPJrhWeCrRmx/JAY=\n-----END PUBLIC KEY-----\n'


In [16]:
server_pub_key = serialization.load_pem_public_key(pub_key_text.encode('ascii'))
server_pub_key

In [17]:
shared_key = private_key_for_client.exchange(server_pub_key)
shared_key

b'}\x15\xed\xb5Muh\xda8\x18<\x9dE\xc1\x07\xb3\xf3\x99M\xb9\xb1\xeaA\xb4\x07\x16<\xb4M\r(-'

In [18]:
# Send the server our public key
r = requests.post(url+"/encrypted/", json={
        'pub_key':client_public_pem.decode('utf-8'),
        'uid':1
    })
assert r.status_code == 200
r.json()

{'encrypted_session_key': 'gAAAAABhTTbIre_Tx6702MH4y45HylrngxgXqi2soW6U7PwPa5RBXROVD29O_TGW7DgJPfsQheyAz_W163ysDY7Q97bKFRwsiq6AYdMvJfF0r5YMD8iE32xVEeTNMuhPECX_IAuoklVd'}

In [19]:
enc_session_key = r.json()['encrypted_session_key']
enc_session_key

'gAAAAABhTTbIre_Tx6702MH4y45HylrngxgXqi2soW6U7PwPa5RBXROVD29O_TGW7DgJPfsQheyAz_W163ysDY7Q97bKFRwsiq6AYdMvJfF0r5YMD8iE32xVEeTNMuhPECX_IAuoklVd'

In [20]:
from cryptography.fernet import Fernet
f = Fernet(base64.urlsafe_b64encode(shared_key))
f

In [21]:
new_session_key = f.decrypt(enc_session_key.encode('utf-8'))
new_session_key

b'KQnBihBYYLavHzGVaGJblQrKR86fw4BKMwOlJ84YGAk='

In [22]:
session_cipher = Fernet(new_session_key)
session_cipher

In [23]:
cipher_text = session_cipher.encrypt(plain_text.encode('utf-8'))
cipher_text

b'gAAAAABhTTbIzpPOwXWypxlt6LB9Eal-zlXKaJmHB4VVo5Ht-YGFQnUeckzSKnIRWdTBxAhLf-bHR-PYq-VvxiSLXghPVDpOp8nlBEJm0SDBAcwR6gYjdWL76kPsYaHxmLmgolSfrjagBqDLHC_7q0UjXagndtkg42aJCOurYHDGzLtfD188RIg36zSyJq6W1yRMOCrFwmH9E7QXQZmNy9bZiSeC4ptWT9puOIvKv2Pob4QI4yw5DkHbxRF7GLsmvzHOh_poXV5dDuN2btSvOxj6Pa1QhSOGRT5CFfq47OaVOOodRPHsTgQ='

In [24]:
r = requests.post(url+"/encrypted/", 
                  json={'cipher_text':cipher_text.decode('utf-8'), })
print(r)
print(r.text)

<Response [200]>
{"decrypted message": "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."}


## Summary
* We can see how data can be encrypted and sent across the Internet. When using symmetric encryption, there needs to be a key exchange. This should never be done in the open, like we did in this example. 

* We introduced GET and POST

* We introduced JSON

* We talked about HTTP status codes

In [25]:
u"\U0001F44F"

'👏'